In [1]:
import numpy as np
import pandas as pd

In [2]:
# read in data
df = pd.read_csv('data.csv')
print(df.keys())

Index(['U_p', 't', 'R_p', 'R_g', 'I_p', 'I_g'], dtype='object')


In [3]:
df

,U_p,t,R_p,R_g,I_p,I_g
0,16.05,0,26.5,26.1,151.8,2.5
1,16.15,105,27.4,27.1,153.2,3.8
2,16.31,454,31.3,29.6,154.7,5.0
3,16.31,845,35.5,38.2,154.7,6.0
4,16.79,1168,39.6,43.8,159.0,3.0
5,19.00,1464,43.8,43.0,181.0,3.0
6,19.05,1807,47.9,43.8,181.0,5.0
7,17.18,2277,52.0,50.8,162.8,5.0
8,17.26,2702,56.1,55.8,163.4,3.0
9,17.30,3150,60.2,59.0,163.7,4.4
